In [257]:
# 1. Initial Data Exploration
import pandas as pd

# Read the CSV file
file_path = '../data/highest_paying_seniorDS_jobs.csv'
df = pd.read_csv(file_path)

# Display Basic Information
print(df.info())
print(df.describe())

# Remove the 'job_schedule_type' & 'job_work_from_home' (Won't be using them)
df.drop(['job_schedule_type','job_work_from_home'], axis=1,inplace=True)

# Check for Missing Values
print('\n')
print(df.isnull().sum())

# Display unique values in categorical columns (check for any unexpected categories)
for col_name in df.select_dtypes(include=['object']).columns:
    print(f"\nUnique Values in {col_name}:")
    print(df[col_name].value_counts()) # print the unique values and their counts (categorical data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1686 entries, 0 to 1685
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   job_title           1686 non-null   object 
 1   job_location        1661 non-null   object 
 2   job_schedule_type   1685 non-null   object 
 3   job_work_from_home  335 non-null    float64
 4   company_name        1686 non-null   object 
 5   salary_year_avg     1686 non-null   float64
dtypes: float64(2), object(4)
memory usage: 79.2+ KB
None
       job_work_from_home  salary_year_avg
count               335.0      1686.000000
mean                  1.0    154050.031829
std                   0.0     50865.050688
min                   1.0     45000.000000
25%                   1.0    125000.000000
50%                   1.0    155000.000000
75%                   1.0    173000.000000
max                   1.0    890000.000000


job_title           0
job_location       25
company_nam

##### Preliminary Remarks 
- salary_year_avg of float64 dtype hence, no data type conversion required. Also, all values are in standardized format (no currency symbols or thousands separators).
- `.info()` shows that the 'job_location' column has some null values (25 entries).
- `.describe()` shows that the mean and median are almost equal and that the IQR and std_dev are almost equal as well.
    - Normal Distribution: Mode=Median=Mean (Symmetry) & IQR = 1.35*std
- Both the min and max values of the dataset clearly indicate that there are outliers in the dataset.

In [258]:
# 2. Handling Missing Values

# Get rows with any null values
rows_with_null = df[df.isnull().any(axis=1)] # Returns a DataFrame containing only the rows where at least one column has a null value
rows_with_null_index = rows_with_null.index # To display the rows that previously had null values after they have been replaced
print(rows_with_null.head(4))
# OR Check for null values in 'job_location'
# rows_with_null_in_column = df[df['job_location'].isnull()]

# Replace NaN values in the 'job_location' column with 'Unknown' to ensure that, when grouping by multiple columns (including 
# 'job_location'), these NaN values won't interfere with the groupby operation, i.e., they won't be treated as unique values.
df.loc[:, 'job_location'] = df['job_location'].fillna('Unknown') # Replace NaN with 'Unknown' only in 'job_location' column (Modifies the original DataFrame)
# Display the updated rows that previously had null values
print("\nAfter replacing null values:")
print(df.loc[rows_with_null_index].head(4)) # The loc accessor is used to select rows by their index labels

                                         job_title job_location  \
4                            Senior Data Scientist          NaN   
8                            Senior Data Scientist          NaN   
26                           Senior Data Scientist          NaN   
51  Senior Data Scientist | Hedge Fund | $200-300k          NaN   

          company_name  salary_year_avg  
4   Algo Capital Group         375000.0  
8   Algo Capital Group         375000.0  
26  Algo Capital Group         325000.0  
51               Orbis         250000.0  

After replacing null values:
                                         job_title job_location  \
4                            Senior Data Scientist      Unknown   
8                            Senior Data Scientist      Unknown   
26                           Senior Data Scientist      Unknown   
51  Senior Data Scientist | Hedge Fund | $200-300k      Unknown   

          company_name  salary_year_avg  
4   Algo Capital Group         375000.0  
8   

- Since we are primarily interested in the salary_year_avg column, it is not necessary to remove or fill the NaN values of the 'job_location' column. 
- There are some duplicate rows in the preceding DataFrame. Duplicate rows will be removed in the next data cleaning step. 
    - The presence of the 25 NaN values in the 'job_location' column will not prevent these rows from being identified as duplicates and removed (except for the first occurrence), due to the way the drop_duplicates() method treats NaN values (they are treated as equal to each other).
    - However, these NaN values would interfere with the groupby operations in the next code cell, as NaN values are treated as unique in such operations. To address this, the NaN values in the 'job_location' column were replaced with 'Unknown'. This replacement allows the three duplicate rows that previously contained NaN values to be grouped together with their identical counterparts. 

In [ ]:
import numpy as np
# 3. Identifying Duplicates

# Identify duplicate rows based on the specified columns (Mark all duplicates as 'True', except for the first occurrence)
duplicate_rows = df[df.duplicated(subset=['job_title', 'job_location', 'company_name', 'salary_year_avg'], keep = 'first')]

# Calculate the total number of rows that will be removed and the number of remaining entries
entries = 1686 # total number of entries
number_of_duplicates = duplicate_rows['salary_year_avg'].count()
print(f"The total number of duplicates (without the first occurrence) is {number_of_duplicates}.")
print(f"Hence, the number of remaining entries will be equal to {entries-number_of_duplicates}.")

# Group the duplicate rows by the specified columns, find the count of rows in each group, 
# and integrate the count of each group into the resulting DataFrame.
duplicate_counts = (
    duplicate_rows.groupby(by=['job_title', 'job_location', 'company_name', 'salary_year_avg'])
    .size() # size() to count all items in each group, including NaN values.
    .reset_index(name='count') # Convert back into Dataframe, reset index, and integrate the count into the DataFrame as a new column named 'count'.
)

# Sort based on the count of rows in each group
sorted_duplicates = duplicate_counts.sort_values(by='count', ascending=False)
# Save the DataFrame to a CSV
sorted_duplicates.to_csv('../data/sorted_duplicates.csv',index=False)

# Group by unique salary values to identify potential duplicate clusters across different salaries, which could skew the distribution.
# Use .sum() to get the total number of duplicate rows for each unique salary value (except for the first occurrences)
clustered_duplicates = (
    sorted_duplicates.groupby('salary_year_avg')
    ['count'].sum() # Returns a Series where the index corresponds to the unique salary values and the values represent the total number of duplicate rows for each unique salary value
    .reset_index() # Converts the Series back into a DataFrame and resets its index (The index of the Series becomes the 'salary_year_avg' column)
    .sort_values(by=['count','salary_year_avg'], ascending=False)    
)
clustered_duplicates.to_csv('../data/clustered_duplicates.csv',index=False)
clustered_duplicates.head(6)

The total number of duplicates (without the first occurrence) is 96.
Hence, the number of remaining entries will be equal to 1590.


,salary_year_avg,count
26,157500.0,11
31,170000.0,9
27,160000.0,7
6,115000.0,6
33,175000.0,5
2,91800.0,3


In [261]:
# Group the salary values into ranges of $10,000

# Create bins for salary ranges of $10,000
bins = range(50000, 360000, 10000)
labels = [f'${i}-${i+9999}' for i in bins[:-1]] # Generates labels for each bin (e.g., '$50000-$59999', '$60000-$69999', etc.). 
# Add a new column for the salary ranges (Use pd.cut() to categorize the 'salary_year_avg' values into the defined ranges.)
clustered_duplicates['salary_range'] = pd.cut(clustered_duplicates['salary_year_avg'], bins=bins, labels=labels, right=False)
# Group by salary_range, sum the counts, reset index, and sort
salary_range_sum = (
    clustered_duplicates.groupby('salary_range', observed=True) # Includes only observed categories
    ['count'].sum()
    .reset_index()
    .sort_values(by=['count','salary_range'], ascending=False)
) 
print(salary_range_sum)

       salary_range  count
9   $170000-$179999     17
7   $150000-$159999     17
3   $110000-$119999     13
8   $160000-$169999     11
4   $120000-$129999      7
5   $130000-$139999      6
12  $200000-$209999      4
11  $190000-$199999      3
6   $140000-$149999      3
2     $90000-$99999      3
14  $250000-$259999      2
13  $210000-$219999      2
10  $180000-$189999      2
17  $340000-$349999      1
16  $320000-$329999      1
15  $310000-$319999      1
1     $70000-$79999      1
0     $50000-$59999      1


In [ ]:
# 3. Remove the duplicate rows while keeping the first occurrence
cleaned_df = (
    df.drop_duplicates(subset=['job_title', 'job_location', 'company_name', 'salary_year_avg'], keep='first')
    .copy() # This ensures that cleaned_df is a new DataFrame, not a view of the original one.
    .reset_index(drop=True) # This ensures that the index is reset to a continuous range after removing duplicates.
)
# 4. Validate Data integrity
assert all(cleaned_df['salary_year_avg'] > 0) , "There are negative salaries!"

# 5. Create Boolean Indicator Column (Used in 4_III.b to extract the highest-paying jobs)
median_salary = cleaned_df['salary_year_avg'].median() # Calculate the median of the distribution
# Use the .loc accessor to safely assign the new column (Ensures that the original DataFrame is modified directly, rather than potentially working on a copy)
cleaned_df.loc[:,'above_median'] = (cleaned_df['salary_year_avg'] >= median_salary) # Select all rows in the 'above_median' column (which will be created)

In [265]:
# 6. Final Data Quality Check
print(cleaned_df.info())
print(cleaned_df.describe())
print('\n')
print(cleaned_df.isnull().sum())

# 7. Document Changes
cleaning_log = [
    "1. Handled missing values (NaN to 'Unknown')",
    "2. Removed duplicates",
    "3. Validated data integrity (salaries > 0)",
    "4. Created Boolean Indicator Column (salaries > median)"
]
with open('../data/data_cleaning_log.txt', 'w') as f: # Opens the file in write mode and closes it after the loop ends
    for step in cleaning_log:  # Loop that iterates over each item in the 'cleaning_log' list
        f.write(f"{step}\n")   # Write each step (string) to the file

# 8. Save Cleaned Data
cleaned_df.to_csv('../data/cleaned_df.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1590 entries, 0 to 1589
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   job_title        1590 non-null   object 
 1   job_location     1590 non-null   object 
 2   company_name     1590 non-null   object 
 3   salary_year_avg  1590 non-null   float64
 4   above_median     1590 non-null   bool   
dtypes: bool(1), float64(1), object(3)
memory usage: 51.4+ KB
None
       salary_year_avg
count      1590.000000
mean     153653.360481
std       50855.750194
min       45000.000000
25%      125000.000000
50%      155000.000000
75%      173000.000000
max      890000.000000


job_title          0
job_location       0
company_name       0
salary_year_avg    0
above_median       0
dtype: int64


Most of the duplicate rows were dispersed around the median of the distribution (Q2=$155,000) and thus, were skewing the distribution in the vicinity of the median. A total number of 96 duplicate rows were removed from the original dataset that used to comprise 1686 entries. These duplicates accounted for a 5.7% error in the original dataset.

These duplicate rows are definitely a limitation of the current project, and they should be analyzed separately in a future project. That is, it should be determined whether the few duplicates are due to multiple identical job postings being posted on the same day or within a short time frame, or due to the same job opening being posted throughout the year at regular intervals to attract multiple candidates.
To distinguish between these two cases, filtering based on the date column (job_posted_date) has to occur.
- If the same job posting reappears every 2-4 months, keep it.
- If the same job posting reappears on the same day, days after the original post, or even a couple weeks after it, keep only the original.